# 🧠 Snowflake Cortex AI_COMPLETE Practice Notebook

## Author: [Prasanna Rajagopal](https://www.linkedin.com/in/prasannarajagopal/)
### Principal Solutions Engineer, Snowflake

This notebook demonstrates the power and flexibility of **Snowflake's** **AI_COMPLETE** function - the most versatile Cortex AI function that can generate responses from text prompts using large language models.

## What You'll Learn

1. **AI_COMPLETE Core Capabilities** - Syntax variants and options
2. **Mimicking Other Cortex AI Functions** - How `AI_COMPLETE` can replicate:
   - `AI_CLASSIFY`
   - `AI_SENTIMENT`
   - `AI_EXTRACT`
   - `AI_SIMILARITY`
3. **Practical Use Cases** - Real-world applications with batch processing

## Prerequisites

- Snowflake account with Cortex AI enabled
- Role with `SNOWFLAKE.CORTEX_USER` database role granted
- A warehouse for compute


---
## Section 1: Setup and Configuration

First, let's set up our session context. Run this cell to configure your warehouse and database.


## Please run the cell titled `SECTION 1: SETUP AND CONFIGURATION` each time you start-up this Notebook.  
### This would ensure that the correct Database and Schema is set for you to execute all the SQL successfully.  
### Cell Title or Variable Name: `SQL_Setup_DB_Schema_WH`

In [ ]:
-- ============================================
-- SECTION 1: SETUP AND CONFIGURATION
-- ============================================

-- Set your warehouse (adjust name as needed)
USE WAREHOUSE COMPUTE_WH;

-- Create a database for this practice session (optional - you can use existing)
CREATE DATABASE IF NOT EXISTS CORTEX_AI_PRACTICE;
USE DATABASE CORTEX_AI_PRACTICE;

-- Create a schema for our demo tables
CREATE SCHEMA IF NOT EXISTS AI_COMPLETE_DEMO;
USE SCHEMA AI_COMPLETE_DEMO;

-- Verify setup
SELECT CURRENT_WAREHOUSE(), CURRENT_DATABASE(), CURRENT_SCHEMA();


---
## Section 2: Create Demo Tables

We'll create **TRANSIENT tables** with sample data for our practice exercises. Transient tables are perfect for temporary/demo data as they don't incur Fail-safe storage costs.

### Table Overview:
| Table | Purpose |
|-------|--------|
| `CUSTOMER_REVIEWS` | Sentiment analysis & classification demos |
| `SUPPORT_TICKETS` | Entity extraction demos |
| `PRODUCT_PAIRS` | Similarity comparison demos |
| `DOCUMENTS` | Summarization & multi-task demos |


In [ ]:
-- ============================================
-- TABLE 1: CUSTOMER_REVIEWS
-- Purpose: Sentiment and classification demos
-- ============================================

CREATE OR REPLACE TRANSIENT TABLE CUSTOMER_REVIEWS (
    review_id INT,
    product_name VARCHAR,
    review_text VARCHAR,
    review_date DATE
);

INSERT INTO CUSTOMER_REVIEWS VALUES
(1, 'Wireless Headphones', 'Amazing sound quality and comfortable fit! Battery lasts forever.', '2024-01-15'),
(2, 'Smart Watch', 'Decent features but the battery drains too quickly. Not worth the price.', '2024-01-18'),
(3, 'Laptop Stand', 'Does what it says. Nothing special but works fine.', '2024-01-20'),
(4, 'Bluetooth Speaker', 'Terrible quality! Stopped working after 2 days. Total waste of money.', '2024-02-01'),
(5, 'USB-C Hub', 'Great build quality and fast data transfer. Love the compact design!', '2024-02-05'),
(6, 'Mechanical Keyboard', 'The typing feel is incredible, but it is quite loud. Good for gaming, bad for office.', '2024-02-08'),
(7, 'Webcam HD', 'Picture quality is okay in good lighting but terrible in low light. Microphone is useless.', '2024-02-10'),
(8, 'Portable Charger', 'Lifesaver! Charged my phone 3 times on a single charge. Highly recommend!', '2024-02-12');

SELECT * FROM CUSTOMER_REVIEWS;


In [ ]:
-- ============================================
-- TABLE 2: SUPPORT_TICKETS
-- Purpose: Entity extraction and classification
-- ============================================

CREATE OR REPLACE TRANSIENT TABLE SUPPORT_TICKETS (
    ticket_id INT,
    customer_name VARCHAR,
    customer_email VARCHAR,
    ticket_content VARCHAR,
    submission_date DATE
);

INSERT INTO SUPPORT_TICKETS VALUES
(101, 'Sarah Johnson', 'sarah.j@email.com', 'My order #ORD-2024-5521 arrived damaged. The screen has a crack. I need a replacement ASAP.', '2024-02-10'),
(102, 'Mike Chen', 'mchen@company.org', 'Cannot login to my account. Getting error code E401. Username: mchen_user. Please help!', '2024-02-11'),
(103, 'Emma Williams', 'emma.w@mail.net', 'Requesting refund for subscription. Account ID: SUB-88432. Charged twice this month.', '2024-02-12'),
(104, 'James Rodriguez', 'j.rodriguez@inbox.com', 'Your mobile app keeps crashing on iOS 17. iPhone 15 Pro. Happens when opening settings.', '2024-02-13'),
(105, 'Lisa Park', 'lisa.park@techmail.io', 'Love your product! Just wanted to say the new update is fantastic. Keep up the great work!', '2024-02-14');

SELECT * FROM SUPPORT_TICKETS;


In [ ]:
-- ============================================
-- TABLE 3: PRODUCT_PAIRS
-- Purpose: Similarity comparison demos
-- ============================================

CREATE OR REPLACE TRANSIENT TABLE PRODUCT_PAIRS (
    pair_id INT,
    text_a VARCHAR,
    text_b VARCHAR
);

INSERT INTO PRODUCT_PAIRS VALUES
(1, 'A portable wireless speaker with Bluetooth 5.0', 'Compact Bluetooth audio device for music on the go'),
(2, 'Running shoes with advanced cushioning', 'A recipe for chocolate cake'),
(3, 'Machine learning algorithm for image recognition', 'Deep learning model for computer vision tasks'),
(4, 'Electric vehicle with 300 mile range', 'Battery-powered car that can travel long distances'),
(5, 'Cloud-based project management software', 'Traditional accounting ledger book'),
(6, 'Organic green tea from Japan', 'Japanese matcha tea leaves');

SELECT * FROM PRODUCT_PAIRS;


In [ ]:
-- ============================================
-- TABLE 4: DOCUMENTS
-- Purpose: Summarization and multi-task demos
-- ============================================

CREATE OR REPLACE TRANSIENT TABLE DOCUMENTS (
    doc_id INT,
    doc_title VARCHAR,
    doc_text VARCHAR
);

INSERT INTO DOCUMENTS VALUES
(1, 'Q4 Sales Report', 'Q4 2024 showed remarkable growth with total revenue reaching $4.2M, a 23% increase from Q3. The Western region led with $1.8M in sales. New product launches contributed 35% of total revenue. Customer retention improved to 89%. Key challenges included supply chain delays affecting 12% of orders.'),
(2, 'Meeting Notes', 'Team meeting on Feb 15, 2024. Attendees: John, Sarah, Mike, Lisa. Decisions: Launch date set for March 1st. Budget approved at $50K. Action items: John to finalize designs by Feb 20. Sarah handles vendor contracts. Next meeting: Feb 22 at 2pm.'),
(3, 'Product Announcement', 'We are excited to announce the release of ProductX 2.0! This major update includes AI-powered recommendations, a redesigned dashboard, 50% faster performance, and enterprise SSO support. Available starting March 15th. Early adopters get 20% discount. Contact sales@company.com for enterprise pricing.');

SELECT * FROM DOCUMENTS;


---
## Section 3: AI_COMPLETE Core Capabilities

AI_COMPLETE is the most flexible Cortex AI function. Let's explore all its capabilities.

### Syntax Overview

```sql
AI_COMPLETE(
    <model>,           -- Required: LLM model name
    <prompt>,          -- Required: Text prompt or prompt object
    <options>          -- Optional: Configuration options
)
```

### Model Used in This Notebook
- `claude-4-sonnet` - Anthropic's powerful model for complex reasoning and generation
- Change the model, if you prefer. 
- Other available models:
`claude-4-opus`
`claude-3-7-sonnet`
`claude-3-5-sonnet`
`deepseek-r1`
`llama3-8b`
`llama3-70b`
`llama3.1-8b`
`llama3.1-70b`
`llama3.1-405b`
`llama3.3-70b`
`llama4-maverick`
`llama4-scout`
`mistral-large`
`mistral-large2`
`mistral-7b`
`mixtral-8x7b`
`openai-gpt-4.1`
`openai-o4-mini`
`snowflake-arctic`
`snowflake-llama-3.1-405b`
`snowflake-llama-3.3-70b`




### 3.1 Basic Text Completion (Single String)

The simplest form - just provide a model name and a text prompt.


In [ ]:
-- ============================================
-- 3.1 BASIC TEXT COMPLETION
-- Simplest form: model + prompt string
-- ============================================

-- Simple question
SELECT AI_COMPLETE(
    'claude-4-sonnet', 
    'Explain quantum computing in one paragraph for a 10-year-old.'
) AS response;


In [ ]:
-- Creative task
SELECT AI_COMPLETE(
    'claude-4-sonnet', 
    'Write a haiku about data engineering.'
) AS haiku;


In [ ]:
-- Code generation
SELECT AI_COMPLETE(
    'claude-4-sonnet', 
    'Write a Python function that calculates the Fibonacci sequence up to n terms.'
) AS code;


### 3.2 Options Parameter Exploration

AI_COMPLETE accepts an options object to fine-tune the response.

| Option | Type | Description |
|--------|------|-------------|
| `temperature` | FLOAT (0.0-1.0) | Controls creativity. Lower = more deterministic, Higher = more creative |
| `max_tokens` | INT | Maximum number of tokens in the response |
| `top_p` | FLOAT (0.0-1.0) | Nucleus sampling - controls diversity |
| `guardrails` | BOOLEAN | Enable/disable safety guardrails |


## Set a `low temperature` for a focused & deterministic response.

In [ ]:
-- =====================================================
-- 3.2a TEMPERATURE CONTROL
-- Low temperature = More focused/deterministic response
-- =====================================================

SELECT AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => 'List 3 benefits of cloud computing.',
    model_parameters => {
        'temperature': 0.1  -- Very low: consistent, focused responses
    }
) AS low_temp_response;


## Set a `high temperature` for a more creative/varied response.

In [ ]:
-- =================================================
-- 3.2b TEMPERATURE CONTROL
-- High temperature = More creative/varied response.
-- =================================================

SELECT AI_COMPLETE(
    'claude-4-sonnet',
    'Write a creative tagline for a coffee shop.',
    {
        'temperature': 0.9  -- High: creative, varied responses
    }
) AS high_temp_response;


## Control the length of the response output using `max_tokens`
### Setting a small value for `max_tokens` can lead to truncated or incomplete responses if the model's full answer exceeds the specified limit.
### Costs for using Cortex AI functions are often calculated based on the number of input and output tokens processed. Using `max_tokens` is one way to manage these charges.

In [ ]:
-- ============================================
-- 3.2 c MAX_TOKENS CONTROL
-- Limit the response length
-- max_tokens = 50
-- Short response
-- ============================================

SELECT AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => 'Explain the theory of relativity.',
    model_parameters => {
        'max_tokens': 50  -- Short response
    }
) AS short_response;


## `top_p` is a hyperparameter that controls the randomness and diversity of the language model's output. 
### It is an alternative to the temperature parameter and defines the set of possible tokens the model can select from at each step of the generation process. 
### `top_p` is a value between 0 and 1 (inclusive). The model only considers the most likely tokens whose cumulative probability exceeds the specified top_p value. The remaining tokens are excluded from consideration.
#### A lower `top_p` value (e.g., 0.1) makes the output more deterministic and focused, as only a small set of very likely tokens are considered.
#### A higher `top_p` value (e.g., 0.9) results in more diverse and potentially creative output, as a larger pool of tokens with varying probabilities are included in the selection process.
#### Set a low `top_p` for a consistent output.  

In [ ]:
-- ============================================
-- 3.2d TOP_P (Nucleus Sampling)
-- Controls diversity of token selection
-- ============================================

SELECT AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => 'Suggest a name for a new tech startup.',
    model_parameters => {
        'top_p': 0.5,      -- Consider only top 50% probability tokens
        'temperature': 0.7
    }
) AS startup_name;


In [ ]:
-- ============================================
-- 3.2e COMBINING MULTIPLE OPTIONS
-- ============================================

SELECT AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => 'Write a professional email declining a meeting invitation.',
    model_parameters => {
        'temperature': 0.3,   -- Keep it professional/consistent
        'max_tokens': 200,    -- Reasonable length for an email
        'top_p': 0.9          -- Allow some variation
    }
) AS professional_email;


In [ ]:
-- ============================================
-- 3.2f GUARDRAILS OPTION
-- Enable safety filters (default is true)
-- ============================================

SELECT AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => 'Explain best practices for secure password storage.',
    model_parameters => {
        'guardrails': TRUE,  -- Enable safety guardrails
        'temperature': 0.2
    }
) AS secure_response;


### 3.3 Prompt Object (Advanced)

For complex interactions, use a **prompt object** with system messages and conversation history.

```sql
AI_COMPLETE(
    <model>,
    {
        'messages': [
            {'role': 'system', 'content': '...'},   -- Set assistant behavior
            {'role': 'user', 'content': '...'},     -- User message
            {'role': 'assistant', 'content': '...'} -- Previous assistant response
        ]
    },
    <options>
)
```


## Create the `EQUITY_RESEARCH_REPORTS` table to try roles in `COMPLETE` function.
### Note:

- `AI_COMPLETE` is the updated version of `COMPLETE` (SNOWFLAKE.CORTEX). For the latest functionality, use `AI_COMPLETE`.
- There are `AI_COMPLETE` examples in this notebook.  


In [ ]:
-- ============================================
-- SECTION 1: SETUP AND CONFIGURATION
-- ============================================

-- Set your warehouse (adjust name as needed)
USE WAREHOUSE COMPUTE_WH;

-- Use DATABASE CORTEX_AI_PRACTICE
USE DATABASE CORTEX_AI_PRACTICE;

-- USE SCHEMA AI_COMPLETE_DEMO
USE SCHEMA AI_COMPLETE_DEMO;

-- Verify setup
SELECT CURRENT_WAREHOUSE(), CURRENT_DATABASE(), CURRENT_SCHEMA();

In [ ]:
-- ============================================
-- Create Transient Table for Equity Research
-- ============================================

CREATE OR REPLACE TRANSIENT TABLE EQUITY_RESEARCH_REPORTS (
    report_id INT AUTOINCREMENT,
    ticker VARCHAR(10),
    company_name VARCHAR(100),
    sector VARCHAR(50),
    market_cap_bn DECIMAL(10,2),
    latest_financials VARCHAR(2000),
    analyst_name VARCHAR(50),
    report_date DATE,
    PRIMARY KEY (report_id)
);

In [ ]:
-- ============================================
-- Insert Simulated Equity Research Data
-- ============================================

INSERT INTO EQUITY_RESEARCH_REPORTS 
    (ticker, company_name, sector, market_cap_bn, latest_financials, analyst_name, report_date)
VALUES
    -- Technology Sector
    ('QNTM', 'QuantumLeap Technologies Inc.', 'Technology', 245.50,
     'Revenue grew 34% YoY to $18.2B driven by cloud infrastructure (+52%) and AI services (+78%). Gross margin expanded 320bps to 68.4%. Operating margin 24.1% vs 21.8% prior year. Free cash flow $4.1B. Announced $2B share buyback. P/E 42x vs sector avg 28x. Guidance raised: FY revenue $74-76B (was $71-73B). Key risk: Enterprise spending slowdown.',
     'Sarah Chen', '2024-11-15'),
    
    ('NEXS', 'NexaSoft Solutions Corp.', 'Technology', 52.30,
     'Revenue declined 8% YoY to $3.1B as legacy software sunset accelerated. Cloud transition 45% complete (was 32%). Subscription ARR grew 28% to $1.8B. Gross margin compressed 180bps to 71.2% due to transition costs. Operating loss $120M vs profit $95M last year. Cash position strong at $2.4B. P/E N/A (negative earnings). Management expects profitability return in Q3 next year.',
     'Michael Torres', '2024-11-18'),

    -- Healthcare Sector
    ('BIONX', 'BioNexus Therapeutics Ltd.', 'Healthcare', 18.70,
     'Phase 3 trial for BNX-401 (rare blood disorder) met primary endpoint with p<0.001. FDA priority review granted, PDUFA date March 2025. Pipeline: 4 drugs in Phase 2, 2 in Phase 1. Cash runway 3.2 years at current burn ($180M/quarter). No revenue yet. Partnership with major pharma rumored. Market opportunity $4.5B for lead drug. Key risk: Binary FDA outcome.',
     'Dr. Emily Watson', '2024-11-20'),

    ('MEDVT', 'MedVantage Health Systems', 'Healthcare', 89.40,
     'Revenue grew 12% YoY to $28.4B. Hospital admissions +8%, outpatient +18%. Labor costs stabilized (-2% YoY per admission). Operating margin 8.2% vs 6.9% prior year. Debt/EBITDA improved to 2.8x from 3.4x. Acquired 3 regional hospital networks for $1.2B. P/E 15x vs sector avg 18x. Dividend yield 2.1%. Key risk: Medicare reimbursement changes.',
     'James Liu', '2024-11-12'),

    -- Financial Sector
    ('FNVA', 'FinNova Capital Holdings', 'Financials', 67.80,
     'Net interest income grew 22% YoY to $4.8B benefiting from rate environment. Loan growth 9% with credit quality stable (NPL 0.8%). Fee income flat as M&A advisory slowed. Efficiency ratio improved to 54% from 58%. CET1 ratio 13.2% (well-capitalized). ROE 14.8% vs 12.1% prior year. P/B 1.4x vs historical avg 1.2x. Dividend raised 8%. Key risk: Credit deterioration if recession.',
     'Amanda Foster', '2024-11-14'),

    -- Consumer Sector  
    ('LUXBR', 'LuxeBrands International', 'Consumer Discretionary', 124.60,
     'Revenue grew 6% YoY to $15.8B. Americas +12%, Europe +4%, Asia -3% (China weakness). Gross margin 72.1% stable. DTC channel now 48% of sales (+6pts YoY). Inventory days improved to 95 from 112. Operating margin 22.4%. Launched 2 new product lines with strong initial sell-through. P/E 28x vs luxury peer avg 25x. Key risk: China consumer recovery timing.',
     'Rachel Kim', '2024-11-16'),

    -- Energy Sector
    ('SLRPW', 'SolarPower Dynamics Corp.', 'Energy', 31.20,
     'Revenue grew 45% YoY to $5.2B driven by utility-scale installations (+62%). Residential segment flat amid rate headwinds. Gross margin 18.4% vs 21.2% (panel price competition). Backlog $12.8B (+35% YoY). IRA benefits contributing $0.45/share. Operating cash flow turned positive ($180M). P/E 85x on depressed margins. Key risk: Interest rate impact on project economics.',
     'David Park', '2024-11-19'),

    -- Industrial Sector
    ('APTS', 'AeroParts Systems Inc.', 'Industrials', 42.10,
     'Revenue grew 18% YoY to $7.6B. Commercial aerospace +24% (Boeing/Airbus recovery), Defense +8%. Backlog at record $18.2B. Gross margin 28.4% (+140bps from mix shift). Supply chain constraints easing. Capex $450M for capacity expansion. P/E 22x vs aerospace supplier avg 20x. Dividend yield 1.4%. Free cash flow conversion 92%. Key risk: OEM production delays.',
     'Thomas Wright', '2024-11-17');

In [ ]:
-- Quick check of inserted data
SELECT ticker, company_name, sector, market_cap_bn 
FROM EQUITY_RESEARCH_REPORTS
ORDER BY sector, market_cap_bn DESC;

## SQL Description: Automated Equity Research Summaries via Cortex

This SQL query generates concise, standardized investment summaries for a list of companies stored in the `EQUITY_RESEARCH_REPORTS` table. It utilizes the `SNOWFLAKE.CORTEX.COMPLETE` function with the `Claude 3.5 Sonnet` model to analyze financial data and produce a structured rating and thesis.

## Key Feature: The `"Roles"` Syntax

- Unlike simple text-in/text-out prompting, this query leverages the chat-structured prompt format (a `JSON` array of message objects). 
- This approach allows for sophisticated instruction tuning and "few-shot" prompting directly within the SQL call.

- The `ARRAY_CONSTRUCT` block builds a conversation history with three distinct components:

### System Persona (role: system)

Purpose: Sets the behavior and rules for the AI.

#### Action: 
Defines the model as a "Senior Equity Research Analyst" and enforces a strict output format (specific Rating keywords like "STRONG BUY" or "HOLD").

### Few-Shot Example (role: user & role: assistant)

- Purpose: Provides a concrete example to guide the model's output style.

#### Action:

The user message simulates a request for Apple (AAPL).

The assistant message shows exactly how the output should look (bolded headers, specific metrics, and formatting). This ensures the model replicates this specific structure for all subsequent rows.

### Dynamic Task (role: user)

Purpose: The actual request for the current row of data.

#### Action: 
It dynamically constructs a prompt by concatenating columns (ticker, company_name, latest_financials) from the source table. The model then generates a summary for this specific company, adhering to the persona and formatting rules defined in the previous messages.

## Output
The query returns the original company details alongside a new column, investment_summary. This column contains the AI-generated analysis (Rating, Thesis, Bull/Bear cases) formatted exactly like the "assistant" example provided in the prompt.

## Note: Comparing `AI_COMPLETE` vs. `SNOWFLAKE.CORTEX.COMPLETE`
- The key difference between these two functions is evolution. `AI_COMPLETE` is the modern, standardized version of the legacy `SNOWFLAKE.CORTEX.COMPLETE` function.

- While both functions ultimately perform the same task—generating text completions using Large Language Models (LLMs)—_Snowflake is standardizing its AI capabilities under the top-level AI_* naming convention.

In [ ]:
-- ============================================
-- Standardized Equity Research Summaries
-- Using SNOWFLAKE.CORTEX.COMPLETE (supports roles)
-- ============================================

SELECT 
    r.ticker,
    r.company_name,
    r.sector,
    r.market_cap_bn,
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-3-5-sonnet',
        ARRAY_CONSTRUCT(
            OBJECT_CONSTRUCT('role', 'system', 'content', 
                'You are a senior equity research analyst. Provide concise investment summaries. Rating must be: STRONG BUY, BUY, HOLD, SELL, or STRONG SELL.'),
            
            OBJECT_CONSTRUCT('role', 'user', 'content', 
                'Summarize: AAPL - Apple Inc. Revenue grew 8% YoY, services up 14%. P/E 28x vs 5yr avg 25x.'),
            OBJECT_CONSTRUCT('role', 'assistant', 'content', 
'**Rating**: HOLD
**Thesis**: Services growth offsets hardware maturation
**Bull Case**: Services expansion, $100B+ cash
**Bear Case**: Above avg P/E, China exposure
**Key Metrics**: Services growth, China revenue'),

            OBJECT_CONSTRUCT('role', 'user', 'content', 
                'Summarize: ' || r.ticker || ' - ' || r.company_name || ' (' || r.sector || ', $' || r.market_cap_bn || 'B). ' || r.latest_financials)
        ),
        {}
    ) AS investment_summary
FROM EQUITY_RESEARCH_REPORTS r
ORDER BY r.sector;

## 📊 Automated Equity Research: Single-String Few-Shot Prompting
This query generates standardized investment summaries using the modern `AI_COMPLETE` function. Unlike the chat-structured approach, this method constructs a single, monolithic prompt string for each record.

### Key Techniques Used:

#### Few-Shot Prompting: 
The prompt includes two concrete examples (Apple and Ford) directly in the text. This "teaches" the model the exact output format and analytical style required (e.g., specific bold headers like **Thesis** and **Bull Case**) before it processes the new data.

#### Dynamic Concatenation: 
The INPUT: section at the end of the prompt is dynamically built using SQL concatenation (`||`), combining the static instructions with the live data (ticker, financials) for the current row.

#### Model Configuration: 
Uses Claude 3.5 Sonnet with a low `temperature` (0.3) to ensure the analysis remains factual, consistent, and strictly adheres to the requested format without "hallucinating" creative styles.

### Output: 
Returns a formatted `investment_summary` string for each company, ready for reporting or dashboard display.

In [ ]:
-- ============================================
-- Standardized Equity Research Summaries
-- Using AI_COMPLETE with Single String Prompt
-- ============================================

SELECT 
    r.ticker,
    r.company_name,
    r.sector,
    r.market_cap_bn,
    AI_COMPLETE(
        model => 'claude-3-5-sonnet',
        prompt => '### INSTRUCTIONS ###
You are a senior equity research analyst at a top investment bank.
Provide concise, actionable investment summaries in the exact format shown below.
Rating must be: STRONG BUY, BUY, HOLD, SELL, or STRONG SELL.

### EXAMPLE 1 ###
INPUT: AAPL - Apple Inc. (Technology, $2.9T). Revenue grew 8% YoY, services up 14%, iPhone flat. Gross margin 45.2%. P/E 28x vs 5yr avg 25x. $100B+ cash. Key risk: China regulatory.

OUTPUT:
**Ticker**: AAPL | **Rating**: HOLD
**Thesis**: Services growth (14% YoY) offsets hardware maturation
**Bull Case**: Services margin expansion, $100B+ cash, AR/VR optionality
**Bear Case**: Trading above 5yr avg P/E, iPhone slowing, China exposure
**Key Metrics**: Services growth rate, China revenue, Gross margin

### EXAMPLE 2 ###
INPUT: F - Ford Motor Co. (Consumer Discretionary, $48B). Revenue flat YoY, EV losses $4.5B, ICE profitable. Inventory 85 days. P/E 6x. Dividend yield 5.2%.

OUTPUT:
**Ticker**: F | **Rating**: HOLD
**Thesis**: Legacy ICE profitability funds EV transition
**Bull Case**: Strong ICE cash flow, 5.2% dividend yield, EV losses peaking
**Bear Case**: EV losses pressuring margins, UAW costs, high inventory
**Key Metrics**: EV unit profitability, Inventory days, ICE margins

### NOW ANALYZE THIS ###
INPUT: ' || r.ticker || ' - ' || r.company_name || ' (' || r.sector || ', $' || r.market_cap_bn || 'B). ' || r.latest_financials || '

OUTPUT:',
        model_parameters => {'temperature': 0.3, 'max_tokens': 1024}
    ) AS investment_summary
FROM EQUITY_RESEARCH_REPORTS r
ORDER BY r.sector;

## Another example of the `role` syntax in the `SNOWFLAKE.CORTEX.COMPLETE` function.

```
[
        {
            'role': 'user',
            'content': 'how does a snowflake get its unique pattern?'
        }
]

In [ ]:
DESC TABLE CUSTOMER_REVIEWS;

In [ ]:
-- ============================================
-- 3.3a SYSTEM PROMPT - Setting Context/Role
-- Using demo table data for practical context
-- ============================================

SELECT 
    cr.review_id,
    cr.product_name,
    cr.review_text,
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-3-5-sonnet',
        ARRAY_CONSTRUCT(
            OBJECT_CONSTRUCT(
                'role', 'system',
                'content', 'You are a SQL expert. Construct a SQL using the information provided.'
            ),
            OBJECT_CONSTRUCT(
                'role', 'user',
                'content', 'I have a CUSTOMER_REVIEWS table with columns: review_id (INT), product_name (VARCHAR), review_text (VARCHAR), review_date (DATE). 
Here is a sample row
- review_id: ' || cr.review_id::VARCHAR || '
- product_name: ' || cr.product_name || '
- review_text: ' || cr.review_text || '
Question: How do I find all reviews for products that contain the word "Bluetooth" in the name?'
            )
        ),
        {}  -- Empty options object - REQUIRED for array/roles format!
    ) AS sql_tutor_response
FROM CUSTOMER_REVIEWS cr
WHERE cr.review_id = 1;

In [ ]:
desc table CUSTOMER_REVIEWS;

In [ ]:
-- ============================================
-- 3.3a SYSTEM PROMPT - Setting Context/Role
-- Using demo table data for practical context
-- ============================================

-- TIP: When using dynamic column data, the simple string format works best
-- Include system instructions directly in the prompt string
SELECT 
    cr.review_id,
    cr.product_name,
    cr.review_text,
    SNOWFLAKE.CORTEX.COMPLETE(
        -- 1. Correct Model ID
        'claude-3-5-sonnet',
        
        -- 2. Construct the Array of Messages
        ARRAY_CONSTRUCT(
            -- System Message
            OBJECT_CONSTRUCT(
                'role', 'system',
                'content', 'You are a helpful SQL tutor. Given the table schema and sample data, write a SQL query to answer the user question. Explain your query briefly.'
            ),
            -- User Message (Dynamically built using OBJECT_CONSTRUCT)
            OBJECT_CONSTRUCT(
                'role', 'user',
                'content', 'I have a CUSTOMER_REVIEWS table with columns: review_id (INT), product_name (VARCHAR), review_text (VARCHAR), review_date (DATE).
Here is a sample row:
- review_id: ' || CAST(cr.review_id AS VARCHAR) || '
- product_name: ' || cr.product_name || '
- review_text: ' || cr.review_text || '

Question: How do I find all reviews for products that contain the word "Bluetooth" in the name?'
            )
        ),
        
        -- 3. Options (Optional, e.g., max_tokens)
        OBJECT_CONSTRUCT('max_tokens', 200)
    ) AS sql_tutor_response
FROM 
    CUSTOMER_REVIEWS cr
WHERE 
    cr.review_id = 1;

In [ ]:
SELECT * FROM CUSTOMER_REVIEWS;

### Set `temperature` and `max_tokens`

In [ ]:
-- ============================================
-- 3.3a SYSTEM PROMPT - Setting Context/Role
-- Using demo table data for practical context
-- ============================================

-- TIP: When using dynamic column data, the simple string format works best
-- Include system instructions directly in the prompt string
SELECT 
    cr.review_id,
    cr.product_name,
    cr.review_text,
    SNOWFLAKE.CORTEX.COMPLETE(
        -- 1. Model Name (Must use a supported ID)
        'claude-3-5-sonnet',
        
        -- 2. The Chat History (Array of Objects)
        ARRAY_CONSTRUCT(
            -- System Message
            OBJECT_CONSTRUCT(
                'role', 'system', 
                'content', 'You are a helpful SQL tutor. Given the table schema and sample data, write a SQL query to answer the user question. Explain your query briefly.'
            ),
            
            -- User Message (Dynamically built with || concatenation)
            OBJECT_CONSTRUCT(
                'role', 'user', 
                'content', 'I have a CUSTOMER_REVIEWS table with columns: review_id (INT), product_name (VARCHAR), review_text (VARCHAR), review_date (DATE).
                            Here is a sample row:
                            - review_id: ' || cr.review_id::VARCHAR || '
                            - product_name: ' || cr.product_name || '
                            - review_text: ' || cr.review_text || '
                            
                            Question: How do I find all reviews for products that contain the word "Bluetooth" in the name?'
            )
        ),
        
        -- 3. Options (Optional configuration like temperature)
        OBJECT_CONSTRUCT('temperature', 0, 'max_tokens', 200)
    ) AS sql_tutor_response
FROM CUSTOMER_REVIEWS cr
WHERE cr.review_id = 1;


In [ ]:
-- ============================================
-- 3.3a-2 SYSTEM PROMPT - Batch SQL Tutoring
-- Generate query explanations for each table
-- ============================================

-- Create a helper table with our schema info
WITH TABLE_INFO AS (
    SELECT 'CUSTOMER_REVIEWS' AS table_name, 
           'review_id INT, product_name VARCHAR, review_text VARCHAR, review_date DATE' AS columns,
           'How do I count reviews per product?' AS question
    UNION ALL
    SELECT 'SUPPORT_TICKETS', 
           'ticket_id INT, customer_name VARCHAR, customer_email VARCHAR, ticket_content VARCHAR, submission_date DATE',
           'How do I find tickets from the last 7 days?'
    UNION ALL
    SELECT 'PRODUCT_PAIRS',
           'pair_id INT, text_a VARCHAR, text_b VARCHAR',
           'How do I select pairs where text_a contains a specific keyword?'
)
SELECT 
    table_name,
    question,
    AI_COMPLETE(
        'claude-4-sonnet',
        '[ROLE: You are a concise SQL tutor. Provide ONLY the SQL query with a one-line comment. No other text.]

Table: ' || table_name || '
Columns: ' || columns || '
Question: ' || question
    ) AS sql_answer
FROM TABLE_INFO;


## 💬 Simulating Multi-Turn Conversations for Deep Insights
This query demonstrates how to perform context-aware analysis using `AI_COMPLETE`. rather than treating the prompt as a simple question-and-answer, it constructs a simulated conversation history within the prompt string.

### Key Concept: 
- The "Conversation History" Pattern Even though this is a single function call, the prompt structure tricks the model into thinking it is in the middle of an ongoing dialogue.
- Treat this conversation pattern as stateless, prompt‑based “pseudo‑conversation”, not a full chat engine.

### Step 1 (Simulated): 
The prompt injects the live data (product_name, review_body) into a "past" User message.

### Step 2 (Hardcoded Context): 
It provides a pre-written "Assistant" response acknowledging the sentiment is "mixed." This sets the stage and tone without requiring the model to actually perform that analysis first.

### Step 3 (The Real Task): 
The prompt ends with a follow-up User question ("What specific product improvements would you suggest?").

Why this matters: By providing this "multi-turn" context, we skip the basic summarization phase and force the model to focus purely on the actionable follow-up (improvement suggestions), while ensuring it still "remembers" the original review text contained in the history.

In [ ]:
-- ============================================
-- 3.3b MULTI-TURN CONVERSATION
-- Simulating conversation with actual review data
-- ============================================

SELECT 
    cr.review_id,
    cr.product_name,
    cr.review_text,
    AI_COMPLETE(
        'claude-4-sonnet',
        '[ROLE: You are a product analyst helping to understand customer feedback. Be concise and insightful.]

[CONVERSATION HISTORY]
User: Here is a customer review for ' || cr.product_name || ': "' || cr.review_text || '". What is the main sentiment?
Assistant: Based on this review, the main sentiment appears to be mixed - there are both positive and negative elements mentioned.
User: What specific product improvements would you suggest based on this feedback?

[YOUR RESPONSE]:'
    ) AS improvement_suggestions
FROM CUSTOMER_REVIEWS cr
WHERE cr.review_id IN (2, 6, 7);


In [ ]:
-- ============================================
-- 3.3c PERSONA-BASED RESPONSES
-- Different personas responding to same ticket
-- ============================================

-- Compare how different personas respond to the same support ticket
WITH TICKET_DATA AS (
    SELECT ticket_id, customer_name, ticket_content 
    FROM SUPPORT_TICKETS 
    WHERE ticket_id = 101  -- Damaged order ticket
)
SELECT 
    'Formal Support Agent' AS persona,
    t.ticket_id,
    AI_COMPLETE(
        'claude-4-sonnet',
        '[ROLE: You are a formal customer support agent. Use professional language, apologize for inconvenience, and provide clear next steps. Keep response under 75 words.]

Customer: ' || t.customer_name || '
Ticket: ' || t.ticket_content
    ) AS response
FROM TICKET_DATA t

UNION ALL

SELECT 
    'Friendly Support Agent' AS persona,
    t.ticket_id,
    AI_COMPLETE(
        'claude-4-sonnet',
        '[ROLE: You are a warm, friendly support agent who uses casual language and empathizes with customers. Show you genuinely care. Keep response under 75 words.]

Customer: ' || t.customer_name || '
Ticket: ' || t.ticket_content
    ) AS response
FROM TICKET_DATA t;


---
## Section 4: Mimicking Other Cortex AI Functions

AI_COMPLETE is so flexible that it can replicate the behavior of other specialized Cortex AI functions through careful prompt engineering!

| Native Function | Purpose | Can AI_COMPLETE do it? |
|-----------------|---------|------------------------|
| AI_CLASSIFY | Categorize text into labels | ✅ Yes |
| AI_SENTIMENT | Analyze sentiment | ✅ Yes |
| AI_EXTRACT | Extract structured data | ✅ Yes |
| AI_SIMILARITY | Compare text similarity | ✅ Yes |


### 4.1 Mimicking AI_CLASSIFY

**AI_CLASSIFY** categorizes text into predefined categories.

```sql
-- Native AI_CLASSIFY syntax:
AI_CLASSIFY(<text>, <categories_array>)
```

Let's replicate this with AI_COMPLETE!


In [ ]:
-- ============================================
-- 4.1a NATIVE AI_CLASSIFY (for comparison)
-- ============================================

SELECT 
    review_text,
    AI_CLASSIFY(
        review_text, 
        ['positive', 'negative', 'neutral']
    ) AS native_classification
FROM CUSTOMER_REVIEWS
LIMIT 3;


## 🏷️ Custom Classification with AI_COMPLETE
This query demonstrates how to perform single-label text classification using the general-purpose `AI_COMPLETE` function, effectively mimicking the specialized `SNOWFLAKE.CORTEX.CLASSIFY_TEXT` (or `AI_CLASSIFY`) function but with greater customizability.

### How It Works:

#### Strict Instruction Tuning: 
The prompt uses a rigid instruction block ([INSTRUCTION: ...]) to constrain the Large Language Model (LLM). It explicitly demands only the category name as output ("positive", "negative", or "neutral") and forbids any conversational filler or explanation.

#### Data Injection: 
The review_body is dynamically appended to the instruction string using SQL concatenation (`||`).

#### Post-Processing: 
The `TRIM()` function is wrapped around the result to remove any accidental whitespace or newlines the model might generate, ensuring the output is clean and join-ready.

#### Use Case: 
While `AI_CLASSIFY` is optimized for this task, using `AI_COMPLETE` allows for more complex logic if needed later—such as handling edge cases, adding custom categories on the fly, or asking the model to "explain its reasoning" in a subsequent step.

In [ ]:
-- ============================================
-- 4.1b AI_COMPLETE MIMICKING AI_CLASSIFY
-- Single classification with just the label
-- ============================================

SELECT 
    review_text,
    TRIM(AI_COMPLETE(
        'claude-4-sonnet',
        '[INSTRUCTION: Classify the given text into exactly ONE of these categories: positive, negative, neutral. Respond with ONLY the category name, nothing else.]

Text to classify: ' || review_text
    )) AS ai_complete_classification
FROM CUSTOMER_REVIEWS;


In [ ]:
-- ============================================
-- 4.1c AI_COMPLETE WITH CONFIDENCE SCORES
-- AI_COMPLETE can provide MORE than AI_CLASSIFY!
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    review_id,
    product_name,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Classify the sentiment of this text and provide confidence score with brief reasoning: ' || review_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'category': {'type': 'string', 'enum': ['positive', 'negative', 'neutral']},
                    'confidence': {'type': 'number'},
                    'reasoning': {'type': 'string'}
                },
                'required': ['category', 'confidence', 'reasoning']
            }
        }
    ) AS classification_with_confidence
FROM CUSTOMER_REVIEWS
LIMIT 5;


In [ ]:
-- ============================================
-- 4.1d MULTI-LABEL CLASSIFICATION
-- Classify support tickets into multiple categories
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    ticket_id,
    ticket_content,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Classify this support ticket into ALL applicable categories (billing, technical, shipping, account, feedback, urgent) and identify the primary category: ' || ticket_content,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'categories': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'primary': {'type': 'string'}
                },
                'required': ['categories', 'primary']
            }
        }
    ) AS multi_label_classification
FROM SUPPORT_TICKETS;


### 4.2 Mimicking AI_SENTIMENT

**AI_SENTIMENT** analyzes text sentiment, optionally for specific categories/aspects.

```sql
-- Native AI_SENTIMENT syntax:
AI_SENTIMENT(<text>, [<category1>, <category2>, ...])
```

Returns: `positive`, `negative`, `neutral`, `mixed`, or `unknown`


In [ ]:
-- ============================================
-- 4.2a NATIVE AI_SENTIMENT (for comparison)
-- Overall sentiment
-- ============================================

SELECT 
    review_text,
    AI_SENTIMENT(review_text) AS native_sentiment
FROM CUSTOMER_REVIEWS
LIMIT 3;


In [ ]:
-- ============================================
-- 4.2b NATIVE AI_SENTIMENT WITH CATEGORIES
-- Aspect-based sentiment
-- ============================================

SELECT 
    review_body,
    AI_SENTIMENT(
        review_body, 
        ['quality', 'price', 'durability']
    ) AS native_aspect_sentiment
FROM CUSTOMER_REVIEWS
LIMIT 3;


In [ ]:
-- ============================================
-- 4.2c AI_COMPLETE MIMICKING AI_SENTIMENT
-- Overall sentiment analysis
-- ============================================

SELECT 
    review_text,
    AI_COMPLETE(
        'claude-4-sonnet',
        '[INSTRUCTION: Analyze the sentiment. Respond with ONLY one word: positive, negative, neutral, or mixed. No explanation.]

Text: ' || review_text
    ) AS ai_complete_sentiment
FROM CUSTOMER_REVIEWS;


In [ ]:
-- ============================================
-- 4.2d AI_COMPLETE WITH ASPECT-BASED SENTIMENT
-- Mimicking AI_SENTIMENT with categories
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    review_id,
    product_name,
    review_text,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Analyze sentiment for each aspect (overall, quality, price, durability). Use values: positive, negative, neutral, mixed, unknown (if not mentioned). Review: ' || review_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'categories': {
                        'type': 'array',
                        'items': {
                            'type': 'object',
                            'properties': {
                                'name': {'type': 'string', 'enum': ['overall', 'quality', 'price', 'durability']},
                                'sentiment': {'type': 'string', 'enum': ['positive', 'negative', 'neutral', 'mixed', 'unknown']}
                            },
                            'required': ['name', 'sentiment']
                        }
                    }
                },
                'required': ['categories']
            }
        }
    ) AS aspect_sentiment
FROM CUSTOMER_REVIEWS;



In [ ]:
-- ============================================
-- 4.2d-2 STRUCTURED OUTPUTS (response_format)
-- REQUIRES ALL NAMED PARAMETERS
-- ============================================

-- KEY: When using response_format, you MUST use named parameters
-- for ALL arguments (model =>, prompt =>, response_format =>)
-- DO NOT mix positional and named arguments!

SELECT AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => 'Analyze the sentiment for this product review: Amazing sound quality and comfortable fit! Battery lasts forever.',
    response_format => {
        'type': 'json',
        'schema': {
            'type': 'object',
            'properties': {
                'sentiment_categories': {
                    'type': 'object',
                    'properties': {
                        'overall': {'type': 'string'},
                        'quality': {'type': 'string'},
                        'comfort': {'type': 'string'},
                        'battery': {'type': 'string'}
                    },
                    'required': ['overall', 'quality', 'comfort', 'battery']
                }
            }
        }
    }
) AS structured_sentiment;


In [ ]:
-- ============================================
-- 4.2d-3 STRUCTURED OUTPUTS WITH TABLE DATA
-- Named parameters work with dynamic prompts!
-- ============================================

-- Structured output with column concatenation works perfectly
-- when using ALL named parameters
SELECT 
    review_id,
    product_name,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Analyze sentiment for this review: ' || review_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'sentiment_categories': {
                        'type': 'object',
                        'properties': {
                            'overall': {'type': 'string'},
                            'quality': {'type': 'string'},
                            'price': {'type': 'string'},
                            'durability': {'type': 'string'}
                        },
                        'required': ['overall', 'quality', 'price', 'durability']
                    }
                }
            }
        }
    ) AS structured_sentiment
FROM CUSTOMER_REVIEWS;


In [ ]:
-- ============================================
-- 4.2e ENHANCED SENTIMENT WITH SCORE
-- AI_COMPLETE can provide numeric scores!
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    review_id,
    product_name,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Analyze sentiment with a score from -1.0 (very negative) to 1.0 (very positive) and extract key phrases: ' || review_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'sentiment': {'type': 'string', 'enum': ['positive', 'negative', 'neutral', 'mixed']},
                    'score': {'type': 'number'},
                    'key_phrases': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    }
                },
                'required': ['sentiment', 'score', 'key_phrases']
            }
        }
    ) AS enhanced_sentiment
FROM CUSTOMER_REVIEWS;


### 4.3 Mimicking AI_EXTRACT

**AI_EXTRACT** pulls structured data from unstructured text.

```sql
-- Native AI_EXTRACT syntax:
AI_EXTRACT(<text>, <fields_array>)
```


In [ ]:
-- ============================================
-- 4.3a NATIVE AI_EXTRACT (for comparison)
-- ============================================

SELECT 
    ticket_content,
    AI_EXTRACT(
        ticket_content, 
        ['order_number', 'issue_type', 'urgency']
    ) AS native_extraction
FROM SUPPORT_TICKETS
LIMIT 3;


In [ ]:
-- ============================================
-- 4.3b AI_COMPLETE MIMICKING AI_EXTRACT
-- Basic entity extraction
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    ticket_id,
    ticket_content,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Extract the order number (if any), issue type, and urgency level from this support ticket: ' || ticket_content,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'order_number': {'type': 'string'},
                    'issue_type': {'type': 'string'},
                    'urgency': {'type': 'string', 'enum': ['low', 'medium', 'high']}
                },
                'required': ['issue_type', 'urgency']
            }
        }
    ) AS extracted_fields
FROM SUPPORT_TICKETS;


In [ ]:
-- ============================================
-- 4.3d EXTRACTING FROM DOCUMENTS
-- Meeting notes and reports
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    doc_title,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Extract dates, people mentioned, numbers (monetary/percentages/other), action items, and key decisions from this document: ' || doc_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'dates': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'people': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'numbers': {
                        'type': 'object',
                        'properties': {
                            'monetary': {'type': 'array', 'items': {'type': 'string'}},
                            'percentages': {'type': 'array', 'items': {'type': 'string'}},
                            'other': {'type': 'array', 'items': {'type': 'string'}}
                        }
                    },
                    'action_items': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'key_decisions': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    }
                },
                'required': ['dates', 'people', 'action_items', 'key_decisions']
            }
        }
    ) AS document_extraction
FROM DOCUMENTS;


In [ ]:
-- ============================================
-- 4.3c ADVANCED EXTRACTION WITH VALIDATION
-- Extract multiple entity types with context
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    ticket_id,
    customer_name,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Extract all identifiable entities, technical details, request type, and priority from this support ticket: ' || ticket_content,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'identifiers': {
                        'type': 'object',
                        'properties': {
                            'order_id': {'type': 'string'},
                            'account_id': {'type': 'string'},
                            'username': {'type': 'string'},
                            'error_code': {'type': 'string'}
                        }
                    },
                    'technical_details': {
                        'type': 'object',
                        'properties': {
                            'device': {'type': 'string'},
                            'os_version': {'type': 'string'},
                            'app_feature': {'type': 'string'}
                        }
                    },
                    'request_type': {'type': 'string', 'enum': ['refund', 'replacement', 'support', 'feedback', 'other']},
                    'priority': {'type': 'string', 'enum': ['low', 'medium', 'high', 'critical']}
                },
                'required': ['request_type', 'priority']
            }
        }
    ) AS comprehensive_extraction
FROM SUPPORT_TICKETS;


In [ ]:
-- ============================================
-- 4.3d EXTRACTING FROM DOCUMENTS
-- Meeting notes and reports
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    doc_title,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Extract dates, people mentioned, numbers (monetary/percentages/other), action items, and key decisions from this document: ' || doc_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'dates': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'people': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'numbers': {
                        'type': 'object',
                        'properties': {
                            'monetary': {'type': 'array', 'items': {'type': 'string'}},
                            'percentages': {'type': 'array', 'items': {'type': 'string'}},
                            'other': {'type': 'array', 'items': {'type': 'string'}}
                        }
                    },
                    'action_items': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'key_decisions': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    }
                },
                'required': ['dates', 'people', 'action_items', 'key_decisions']
            }
        }
    ) AS document_extraction
FROM DOCUMENTS;


### 4.4 Mimicking AI_SIMILARITY

**AI_SIMILARITY** compares semantic similarity between two texts, returning a score from 0 to 1.

```sql
-- Native AI_SIMILARITY syntax:
AI_SIMILARITY(<text1>, <text2>)
```

AI_COMPLETE can provide similarity analysis with explanations!


In [ ]:
-- ============================================
-- 4.4a NATIVE AI_SIMILARITY (for comparison)
-- ============================================

SELECT 
    pair_id,
    text_a,
    text_b,
    AI_SIMILARITY(text_a, text_b) AS native_similarity_score
FROM PRODUCT_PAIRS;


In [ ]:
-- ============================================
-- 4.4b AI_COMPLETE MIMICKING AI_SIMILARITY
-- Similarity with numeric score
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    pair_id,
    text_a,
    text_b,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Compare semantic similarity (0.0=unrelated, 1.0=identical) between Text A: ' || text_a || ' and Text B: ' || text_b,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'similarity_score': {'type': 'number'},
                    'relationship': {'type': 'string', 'enum': ['identical', 'very_similar', 'similar', 'somewhat_related', 'unrelated']}
                },
                'required': ['similarity_score', 'relationship']
            }
        }
    ) AS ai_complete_similarity
FROM PRODUCT_PAIRS;


In [ ]:
-- ============================================
-- 4.4c ENHANCED SIMILARITY WITH EXPLANATION
-- AI_COMPLETE can explain WHY texts are similar!
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    pair_id,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Analyze semantic similarity, shared concepts, differences, and provide explanation for Text A: ' || text_a || ' and Text B: ' || text_b,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'similarity_score': {'type': 'number'},
                    'shared_concepts': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'differences': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'explanation': {'type': 'string'}
                },
                'required': ['similarity_score', 'shared_concepts', 'differences', 'explanation']
            }
        }
    ) AS detailed_similarity
FROM PRODUCT_PAIRS;


---
## Section 5: Practical Use Cases

Let's combine everything into real-world scenarios using our demo tables.


### 5.1 Customer Feedback Analysis Pipeline

Process all customer reviews with classification, sentiment, and key insights in a single query.


In [ ]:
-- ============================================
-- 5.1 COMPREHENSIVE REVIEW ANALYSIS
-- All-in-one analysis using AI_COMPLETE
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    review_id,
    product_name,
    review_date,
    review_text,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Comprehensively analyze this product review for sentiment, aspects, key points, and recommended action: ' || review_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'sentiment': {'type': 'string', 'enum': ['positive', 'negative', 'neutral', 'mixed']},
                    'sentiment_score': {'type': 'number'},
                    'category': {'type': 'string', 'enum': ['praise', 'complaint', 'suggestion', 'question', 'neutral']},
                    'aspects': {
                        'type': 'object',
                        'properties': {
                            'quality': {'type': 'string'},
                            'price': {'type': 'string'},
                            'usability': {'type': 'string'}
                        }
                    },
                    'key_points': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'recommended_action': {'type': 'string', 'enum': ['none', 'follow_up', 'urgent_response', 'feature_request']},
                    'summary': {'type': 'string'}
                },
                'required': ['sentiment', 'sentiment_score', 'category', 'key_points', 'recommended_action', 'summary']
            }
        }
    ) AS comprehensive_analysis
FROM CUSTOMER_REVIEWS;


### 5.2 Support Ticket Triage System

Automatically categorize, prioritize, and route support tickets.


In [ ]:
-- ============================================
-- 5.2 SUPPORT TICKET TRIAGE
-- Automated ticket processing
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    ticket_id,
    customer_name,
    customer_email,
    submission_date,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Triage this support ticket - categorize, prioritize, extract IDs, suggest team, and summarize: ' || ticket_content,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'category': {'type': 'string', 'enum': ['billing', 'technical', 'shipping', 'account', 'feedback', 'other']},
                    'subcategory': {'type': 'string'},
                    'priority': {'type': 'string', 'enum': ['low', 'medium', 'high', 'critical']},
                    'sentiment': {'type': 'string', 'enum': ['frustrated', 'neutral', 'positive']},
                    'extracted_ids': {
                        'type': 'object',
                        'properties': {
                            'order': {'type': 'string'},
                            'account': {'type': 'string'},
                            'error_code': {'type': 'string'}
                        }
                    },
                    'suggested_team': {'type': 'string', 'enum': ['billing_team', 'tech_support', 'shipping_dept', 'account_mgmt', 'customer_success']},
                    'auto_response_possible': {'type': 'boolean'},
                    'summary': {'type': 'string'}
                },
                'required': ['category', 'priority', 'sentiment', 'suggested_team', 'auto_response_possible', 'summary']
            }
        }
    ) AS triage_result
FROM SUPPORT_TICKETS;


### 5.3 Document Summarization

Summarize documents with different formats and lengths.


In [ ]:
-- ============================================
-- 5.3a EXECUTIVE SUMMARY
-- Brief, high-level summary
-- ============================================

SELECT 
    doc_id,
    doc_title,
    AI_COMPLETE(
        'claude-4-sonnet',
        '[INSTRUCTION: Create an executive summary in 2-3 sentences. Focus on key outcomes and decisions.]

Document: ' || doc_text
    ) AS executive_summary
FROM DOCUMENTS;


In [ ]:
-- ============================================
-- 5.3b STRUCTURED SUMMARY
-- Bullet-point format with key details
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    doc_id,
    doc_title,
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Summarize this document with key points, metrics, action items, and next steps: ' || doc_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'title': {'type': 'string'},
                    'type': {'type': 'string', 'enum': ['report', 'meeting_notes', 'announcement', 'other']},
                    'key_points': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'metrics': {
                        'type': 'array',
                        'items': {
                            'type': 'object',
                            'properties': {
                                'name': {'type': 'string'},
                                'value': {'type': 'string'}
                            }
                        }
                    },
                    'action_items': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'next_steps': {'type': 'string'}
                },
                'required': ['title', 'type', 'key_points', 'action_items', 'next_steps']
            }
        }
    ) AS structured_summary
FROM DOCUMENTS;


### 5.4 Data Enrichment

Add AI-generated metadata to existing data.


In [ ]:
-- ============================================
-- 5.4 DATA ENRICHMENT
-- Add tags, categories, and metadata
-- Using STRUCTURED OUTPUT for guaranteed JSON
-- ============================================

SELECT 
    review_id,
    product_name,
    review_text,
    -- Generate searchable tags
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Generate lowercase metadata tags, topics, and product features mentioned (max 5 each) for Product: ' || product_name || ' Review: ' || review_text,
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'tags': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'topics': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    },
                    'product_features_mentioned': {
                        'type': 'array',
                        'items': {'type': 'string'}
                    }
                },
                'required': ['tags', 'topics', 'product_features_mentioned']
            }
        }
    ) AS generated_metadata
FROM CUSTOMER_REVIEWS;


### 5.5 Multi-Language Translation

Translate text to multiple languages.


In [ ]:
-- ============================================
-- 5.5 TRANSLATION
-- Translate reviews to multiple languages
-- ============================================

SELECT 
    review_id,
    review_text AS original_english,
    AI_COMPLETE(
        'claude-4-sonnet',
        'Translate the following text to Spanish. Return ONLY the translation, nothing else: ' || review_text,
        {'temperature': 0.3}
    ) AS spanish_translation,
    AI_COMPLETE(
        'claude-4-sonnet',
        'Translate the following text to French. Return ONLY the translation, nothing else: ' || review_text,
        {'temperature': 0.3}
    ) AS french_translation
FROM CUSTOMER_REVIEWS
LIMIT 3;


### 5.6 Response Generation

Generate personalized responses to customer feedback.


In [ ]:
-- ============================================
-- 5.6 AUTO-GENERATE RESPONSES
-- Create personalized customer responses
-- ============================================

SELECT 
    ticket_id,
    customer_name,
    ticket_content,
    AI_COMPLETE(
        'claude-4-sonnet',
        '[ROLE: You are a friendly customer support agent for TechCorp. Write a helpful, empathetic response. Include: 1) Acknowledge their issue, 2) Provide solution/next steps, 3) Professional sign-off. Keep it under 100 words. Use the customer name.]

Customer Name: ' || customer_name || '
Ticket: ' || ticket_content
    ) AS suggested_response
FROM SUPPORT_TICKETS;


---
## Section 6: Cleanup (Optional)

Run this cell to clean up the demo tables when you're done practicing.


In [ ]:
-- ============================================
-- CLEANUP (Optional)
-- Remove demo tables and schema
-- ============================================

-- Uncomment the lines below to clean up:
-- DROP TABLE IF EXISTS CUSTOMER_REVIEWS;
-- DROP TABLE IF EXISTS SUPPORT_TICKETS;
-- DROP TABLE IF EXISTS PRODUCT_PAIRS;
-- DROP TABLE IF EXISTS DOCUMENTS;
-- DROP TABLE IF EXISTS EQUITY_RESEARCH_REPORTS;
-- DROP SCHEMA IF EXISTS AI_COMPLETE_DEMO;
-- DROP DATABASE IF EXISTS CORTEX_AI_PRACTICE;

SELECT 'Cleanup commands are commented out. Uncomment to execute.' AS message;


---
## Summary

### What We Covered

| Topic | Key Learnings |
|-------|---------------|
| **Basic AI_COMPLETE** | Simple text prompts, model selection |
| **Options** | temperature, max_tokens, top_p, guardrails |
| **Prompt Objects** | System prompts, multi-turn conversations |
| **Structured Outputs** | `response_format` with JSON schema for guaranteed valid JSON |
| **Mimicking AI_CLASSIFY** | Classification with confidence scores |
| **Mimicking AI_SENTIMENT** | Aspect-based sentiment with scores |
| **Mimicking AI_EXTRACT** | Structured entity extraction |
| **Mimicking AI_SIMILARITY** | Similarity with explanations |
| **Practical Uses** | Pipelines, triage, summarization, enrichment |

### Key Takeaways

1. **AI_COMPLETE is the Swiss Army knife** of Cortex AI - it can do what other functions do, and more
2. **Structured Outputs are powerful** - use `response_format` with JSON schema for guaranteed valid JSON
3. **Named parameters required** - when using `response_format`, ALL arguments must use `=>` syntax
4. **Temperature control** - use low values (0-0.3) for consistent outputs, higher for creativity
5. **Native functions are optimized** - use AI_CLASSIFY, AI_SENTIMENT, etc. when you need their specific output format

### Structured Output Syntax (Important!)

```sql
AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => 'Your prompt here: ' || column_data,
    response_format => {
        'type': 'json',
        'schema': {
            'type': 'object',
            'properties': {
                'field_name': {'type': 'string'},
                'numeric_field': {'type': 'number'},
                'array_field': {'type': 'array', 'items': {'type': 'string'}}
            },
            'required': ['field_name']
        }
    }
)
```

### Next Steps

- Experiment with different models (llama3.1-70b, mistral-large2)
- Try image analysis with AI_COMPLETE (requires staged images)
- Build production pipelines combining multiple AI functions
- Explore Cortex Search for RAG applications
